In [1]:
#!pip install yt_dlp
#!pip install opencv-python

In [2]:
import yt_dlp
import cv2
import re

In [3]:
def CheckResolution(formats:"yt_dlp.YoutubeDL.extract_info.get('formats')", resolution:str = "best") -> list[dict]:
    
    '''

    Check if the resolution supplied is avilable, otherwise return False

    Parameters
    ----------

      formats:list[dict]
        list of diccionaries obtained from yt_dlp.YoutubeDL.extract_info.get('formats')

      resolution:str, default "best"
        string indicating to check the resolution, e.g "144p", "360p", "1080p60", etc 


    Returns
      str|bool: str indicating resolution if available, otherwise returns bool False

    '''

    if resolution != "best":

        matches = [resolution for f in formats if f.get("format_note", None) == resolution]
        matches_len = len(matches)

        return resolution if matches_len else False

    max_resolution_available = {"res": "0p"}

    for f in formats:
    
        current_res = f.get("format_note", "0p")
        
        # Only retrieving if metadata is describing some numeric value
        if bool(re.match("^\d+", current_res)):
            current_pixels = re.split("[A-Za-z]", current_res)[0]
            current_pixels = int(current_pixels)
            
            before_pixels = re.split("[A-Za-z]", max_resolution_available["res"])[0]
            before_pixels = int(before_pixels)
            
            max_res = current_res if current_pixels > before_pixels else max_resolution_available["res"] 
            max_resolution_available["res"] = max_res 
            
    return max_resolution_available["res"]

def GetMetaVideo(formats:"yt_dlp.YoutubeDL.extract_info.get('formats')", resolution, which:int=-1):

    url_options = [f for f in formats if f.get("format_note") == resolution]

    return url_options[which]

def RemoveEmojis(string:str):
    
    return re.sub(r'[^\x00-\x7F]+',' ', string)

In [10]:
# Load model cascade
face_cascade_frontal = cv2.CascadeClassifier('data/models/haarcascade_frontalface_default.xml')

In [5]:
# Initializing youtube video object
video_url = "https://www.youtube.com/watch?v=5NjJLFI_oYs"
ydl = yt_dlp.YoutubeDL({})

# Loading youtube video data
info_dict = ydl.extract_info(video_url, download=False)
video_title = info_dict["title"]
video_title = RemoveEmojis(video_title)

# Get video formats available
formats = info_dict.get('formats', None)

[youtube] Extracting URL: https://www.youtube.com/watch?v=kiC1rPh1C3U
[youtube] kiC1rPh1C3U: Downloading webpage
[youtube] kiC1rPh1C3U: Downloading android player API JSON


In [6]:
is_present_res = CheckResolution(formats, "360p")

if not is_present_res:
    raise Exception("Resolution not available")

last_format = GetMetaVideo(formats, is_present_res, -1)

In [7]:
capture = cv2.VideoCapture(last_format.get("url"))

In [8]:
capture = cv2.VideoCapture(last_format.get("url"))

if not capture.isOpened():
    raise Exception("Can't be possible to open the video")
    
while True:
    ret, frame = capture.read()
    
    if not ret:
        print("Video has ended")
        break
        
    # Gray scale converting
    gray_scale_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
    # Face detection front and profile
    location_face_frontal = face_cascade_frontal.detectMultiScale(gray_scale_frame, 1.2, 5)        
        
    for (x, y, w, h) in location_face_frontal:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
    
    cv2.imshow(video_title, frame)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break
    
capture.release()
cv2.destroyAllWindows()